### **Métodos de Optimización**
#### Prácticas computacionales de "Programación Lineal"

Instrucciones para los Ejercicios

1. **Trabajo en Grupo:**
   - Los ejercicios deben ser resueltos y entregados en grupo.
   - La cantidad de integrantes por grupo será definida el día de la actividad, así como la fecha límite para la entrega.

2. **Uso de Google Colab y Compartir:**
   - Este notebook debe ser copiado al GitHub o Google Drive de alguno de los integrantes del grupo.
   - El grupo será responsable de programar las soluciones, realizar las pruebas y enviar el trabajo final al profesor.

3. **Implementación de los Ejercicios:**
   - Cada ejercicio debe ser implementado de manera que cumpla con los objetivos específicos descritos en cada problema.
   - El código debe devolver claramente la información calculada de acuerdo a lo solicitado.

4. **Calidad del Código:**
   - El código debe ejecutarse sin errores.
   - Es obligatorio incluir **comentarios explicativos** para describir las ideas y conceptos implícitos en el código, facilitando la comprensión de su lógica.

5. **Envío del Trabajo:**
   - Una vez completado, el notebook debe ser enviado a través de Moodle.
   - En caso de dudas, pueden contactarme por correo electrónico a **marcelo.danesi@utec.edu.uy**.

6. **Orientaciones Adicionales:**
   - Asegúrense de que todas las celdas de código hayan sido ejecutadas antes de enviar.
   - Incluyan el nombre completo y correo electrónico de todos los integrantes al inicio del notebook.
   - Si utilizan referencias externas, menciónenlas de forma adecuada.

¡Buena suerte y aprovechen la práctica para consolidar los conceptos de métodos optimización!

#### **Programación Lineal y Método Simplex**



#### **1) Maximización con Simplex (visión computacional)**

Un problema estándar de **maximización lineal** busca
$$
\begin{aligned}
\max\; z &= c^\top x \\
\text{s.a.}\;& A x \le b\\
& x \ge 0.
\end{aligned}
$$

donde $x\in\mathbb{R}^n$ son las variables de decisión, $A\in\mathbb{R}^{m\times n}$ y $b\in\mathbb{R}^m$. En la práctica, las librerías numéricas (como `scipy.optimize.linprog`) resuelven $\min\; c^\top x$. Para resolver una $\max$, transformamos a $\min(-c^\top x)$. El método HiGHS (por defecto en `linprog`) ejecuta una variante robusta (simplex revisado/interior point según el caso), entregando soluciones eficientes incluso para dimensiones medianas-grandes.

##### **Ejemplo: Mezcla de producción (2 variables)**

$$
\begin{aligned}
\max\; z &= 3x_1 + 2x_2 \\
\text{s.a.}\;& 2x_1 + x_2 \le 100 \quad (\text{horas de trabajo})\\
& x_1 + 2x_2 \le 80 \quad (\text{materia prima})\\
& x_1, x_2 \ge 0.
\end{aligned}
$$
**Objetivo**  
Formular el LP en matrices $(c,A_{ub},b_{ub})$, resolver en Python, imprimir $z^*$, $x^*=(x_1^*, x_2^*)$ y analizar qué restricciones quedan activas (con folga cero).

**Nota:**   
En programación lineal, una **restricción está activa** cuando se cumple en forma de igualdad en la solución óptima, es decir, la holgura asociada es igual a cero.  

Por ejemplo, si la restricción es  
$$ 2x_1 + x_2 \leq 100,$$  
y en el óptimo obtenemos $2x_1 + x_2 = 100$, decimos que esta restricción está activa.  

- Cuando una restricción está activa, significa que “consume” completamente el recurso que representa (horas, materia prima, presupuesto, etc.).  
- Por el contrario, si en el óptimo se cumple $2x_1 + x_2 < 100$, la restricción no está activa: queda holgura positiva y el recurso no se agotó.

Analizar cuáles restricciones están activas en el óptimo nos da información importante: nos indica qué recursos son **limitantes** y cuáles no lo son en la solución encontrada.

In [43]:
# --- Sección 1: Maximización con Simplex (SciPy/HiGHS) ---

import numpy as np
from scipy.optimize import linprog

def solve_lp_max(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs"):
    """
    Resuelve max c^T x convirtiéndolo a min (-c)^T x.
    Retorna el objeto res de linprog; imprime z* y x*.
    """
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)   # x >= 0 por defecto
    res = linprog(c=-c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)
    if not res.success:
        print("⚠️ linprog:", res.message)
    z = float(c @ res.x)  # revertimos el signo para el valor máximo
    print("z* =", z)
    print("x* =", res.x)
    # Folgas de las restricciones <= (si existen)
    if A_ub is not None:
        slack = b_ub - A_ub @ res.x
        print("slack (<=) =", slack)
    return res

# Datos del ejemplo (mezcla de producción)
c   = [3, 2]
Aub = [[2, 1],
       [1, 2]]
bub = [100, 80]

res = solve_lp_max(c, A_ub=Aub, b_ub=bub)

z* = 160.0
x* = [40. 20.]
slack (<=) = [0. 0.]



🔎 **Sugerencia:**  
En algunos de los ejercicios, puede ser útil generar matrices o vectores aleatorios para crear nuevos problemas de Programación Lineal y probar el código.  
Podés usar la función `numpy.random` para eso.  
Consultá la documentación oficial de NumPy para ver las distintas formas de generar números aleatorios:  
👉 [https://numpy.org/doc/stable/reference/random/generator.html](https://numpy.org/doc/stable/reference/random/generator.html)


##### **Tarea 1: Modificación de la función objetivo.**

Sustituir $z=3x_1+2x_2$ por $z=4x_1+x_2$; resolver y *explicar* qué vértice del politopo queda activo y por qué.

Recuerde la nota explicativa del ejemplo.

In [44]:
# --- Sección 1: Maximización con Simplex (SciPy/HiGHS) ---

import numpy as np
from scipy.optimize import linprog

def solve_lp_max(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs"):
    """
    Resuelve max c^T x convirtiéndolo a min (-c)^T x.
    Retorna el objeto res de linprog; imprime z* y x*.
    """
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)   # x >= 0 por defecto
    res = linprog(c=-c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)
    if not res.success:
        print("⚠️ linprog:", res.message)
    z = float(c @ res.x)  # revertimos el signo para el valor máximo
    print("z* =", z)
    print("x* =", res.x)
    # Folgas de las restricciones <= (si existen)
    if A_ub is not None:
        slack = b_ub - A_ub @ res.x
        print("slack (<=) =", slack)
    return res

# Datos del ejemplo (mezcla de producción)
c   = [4, 1]     # z=4x+y
Aub = [[2, 1],
       [1, 2]]
bub = [100, 80]

res = solve_lp_max(c, A_ub=Aub, b_ub=bub)

z* = 200.0
x* = [50.  0.]
slack (<=) = [ 0. 30.]


El vértice (50, 0). Este vértice es el punto de intersección de las siguientes restricciones:

    2x​+y ​= 100     (Restricción 1)
    x = 0          (Restricción de no negatividad)

El politopo de soluciones factibles está definido por las siguientes restricciones:

    2x +y ​ ≤ 100
    x​ + 2y ​≤ 80
    x ​≥ 0
    y ​≥ 0

Los vértices de este politopo son:

    (0, 0)  : Intersección de x​=0 y y​=0.             Valor de la función objetivo: z=4(0)+0=0
    (0, 40) : Intersección de x​=0 y x​+2y​=80          Valor de la función objetivo: z=4(0)+40=40
    (40, 20): Intersección de 2x​+y​=100 y x​+2y​=80     Valor de la función objetivo:  z=4(40)+20=180
    (50, 0) : Intersección de 2x​+y​=100 y y​=0         Valor de la función objetivo: z=4(50)+0=200


La solución óptima se encuentra en uno de los vértices del politopo.
Al evaluar la nueva función objetivo (z=4x + y) en cada uno de estos vértices, el valor máximo es 200, que se alcanza en el vértice (50, 0).

Las restricciones activas en este vértice son las que definen el punto:

    2 x​ + y ​= 100 (Restricción 1)
    y ​      =   0 (Restricción de no negatividad)
    
La segunda restricción original (x + 2y ≤ 80) no está activa, ya que al evaluar el punto (50, 0) se obtiene 50+2(0)=50, que es menor que 80, lo que deja un slack de 30.

##### **Tarea 2: Ajuste de restricciones del modelo.**


Agregar la restricción $x_1 \le 50$; resolver y comparar $z^*$ con el caso base, comentando el efecto.

In [45]:
# --- Sección 1: Maximización con Simplex (SciPy/HiGHS) ---

import numpy as np
from scipy.optimize import linprog

def solve_lp_max(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs"):
    """
    Resuelve max c^T x convirtiéndolo a min (-c)^T x.
    Retorna el objeto res de linprog; imprime z* y x*.
    """
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)   # x >= 0 por defecto
    res = linprog(c=-c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)
    if not res.success:
        print("⚠️ linprog:", res.message)
    z = float(c @ res.x)  # revertimos el signo para el valor máximo
    print("z* =", z)
    print("x* =", res.x)
    # Folgas de las restricciones <= (si existen)
    if A_ub is not None:
        slack = b_ub - A_ub @ res.x
        print("slack (<=) =", slack)
    return res

# Datos del ejemplo (mezcla de producción)
c   = [4, 1]
Aub = [[2, 1],
       [1, 2],
       [1, 0]]                  # restriccion 1 *x1 (= 50)
bub = [100, 80,50]              # nueva restriccion (el valor 50 al final del vector)

res = solve_lp_max(c, A_ub=Aub, b_ub=bub)

z* = 200.0
x* = [50. -0.]
slack (<=) = [ 0. 30.  0.]


Para agregar la restricción x1 ≤ 50 debemos expresarla en la forma estándar de A_ub . x <= b_ub.

Esto significa que el vector A_ub debe tener una fila adicional [1, 0] y el vector b_ub debe tener un elemento adicional 50.

Análisis de la Solución:

    Valor Óptimo: El valor de la función objetivo sigue siendo 200.0.
    Solución Óptima : El punto óptimo sigue siendo [50, 0].
    Holgura (slack): La holgura para la nueva restricción [1, 0] es 0, lo que indica que es una restricción activa.

Efecto de la Nueva Restricción:

    El valor óptimo z no cambió. Esto se debe a que la nueva restricción (x_1 ≤  50) es redundante.
    El punto óptimo del problema original (sin la nueva restricción) ya satisfacía esta condición, ya que el valor de x1​ en el óptimo era 50

En términos gráficos, la solución óptima del problema original estaba en el vértice (50, 0). La restricción x1 ≤ 50 no reduce el espacio de soluciones utiles de manera que excluya este vértice.

La nueva restricción solo confirma una propiedad que la solución óptima ya tenía.


##### **Tarea 3: Resolución de un problema de mayor escala.**

 Generar un LP aleatorio grande ($n=50$, $m=100$) con $A\sim U[0,1]$, $b\sim U[20,40]$, $c\sim U[0,1]$; resolver, medir tiempo de cómputo usando `import time` (o similar) y reportar $z^*$.

In [46]:
import numpy as np
from scipy.optimize import linprog
import time

def solve_lp_max(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs"):
    """
    Resuelve max c^T x convirtiéndolo a min (-c)^T x.
    Retorna el objeto res de linprog; imprime z* y x*.
    """
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)
    res = linprog(c=-c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)
    if not res.success:
        print("⚠️ linprog:", res.message)
    z = float(c @ res.x)
    print("z* =", z)
    #print("x* =", res.x)  # Se comenta para no imprimir el vector grande
    return res

# --- Generación del LP aleatorio ---
n = 50  # Número de variables     (columnas de A)
m = 100 # Número de restricciones (filas de A)

# A ~ U[0,1]
A_ub_rand = np.random.uniform(0, 1 + np.finfo(float).eps, size=(m, n))  # + np.finfo(float).eps agrega el minimo valor posible para que el 1
                                                                        # pueda caer en la seleccion del random ya que el intervalo es cerrado

# b ~ U[20,40]
b_ub_rand = np.random.uniform(20, 40 + np.finfo(float).eps, size=m)     # + np.finfo(float).eps agrega el minimo valor posible para que el 40
                                                                        # pueda caer en la seleccion del random ya que el intervalo es cerrado

# c ~ U[0,1]
c_rand = np.random.uniform(0, 1 + np.finfo(float).eps, size=n)          # + np.finfo(float).eps agrega el minimo valor posible para que el 1
                                                                        # pueda caer en la seleccion del random ya que el intervalo es cerrado


print("Generando un LP aleatorio:")
print(f"Número de variables (n): {n}")
print(f"Número de restricciones (m): {m}")



# --- Resolución y medición del tiempo ---
start_time = time.time()
res_rand = solve_lp_max(c_rand, A_ub=A_ub_rand, b_ub=b_ub_rand)
end_time = time.time()

# --- Reporte de resultados ---
print("--------------------------------------------------------------------")
print(f"Tiempo de cómputo: {end_time - start_time:.4f} segundos")
print(f"Valor óptimo (z*): {res_rand.fun * -1:.4f}")

Generando un LP aleatorio:
Número de variables (n): 50
Número de restricciones (m): 100
z* = 35.65398089127282
--------------------------------------------------------------------
Tiempo de cómputo: 0.0126 segundos
Valor óptimo (z*): 35.6540


#### **2) Minimización (primal con $\ge$ vs. dual)**

Para
$$
\begin{aligned}
\min\; z &= c^\top x \\
\text{s.a.}\;& A x \ge b\\
& x \ge 0,
\end{aligned}
$$
hay dos estrategias prácticas:  
1. transformar a $-A x \le -b$ y usar `linprog` directamente;
2. construir el **dual** (que es $\max$ con $\le$) y resolverlo con la misma rutina de maximización. La equivalencia $C_{\min}=P_{\max}$ (dualicidad fuerte, bajo condiciones regulares) permite verificar resultados y reconstruir la solución primal por *complementariedad*.

##### **Ejemplo: Conversión directa ($Ax \ge b \to -Ax \le -b$).**

$$
\begin{aligned}
\min\; C &= 2x + y + 2z \\
\text{s.a. }& x + y + z \ge 4,\\
& 2x + y + 3z \ge 6,\\
& x,y,z \ge 0.
\end{aligned}
$$

**Objetivo**   
- Transformar las restricciones $\ge$ a $\le$ multiplicando por $-1$.
- Resolver en Python e interpretar $C^*$ y $x^*$.

*Opcional:* Montar y resolver el dual para verificar $C_{\min}=P_{\max}$.

In [47]:
# --- Sección 2: Minimización (Ax >= b -> -Ax <= -b) ---

import numpy as np
from scipy.optimize import linprog

def solve_lp_min(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs"):
    """
    Resuelve min c^T x en forma directa (<= , =).
    Retorna el objeto res de linprog; imprime C* y x*.
    """
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)
    res = linprog(c=c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)
    if not res.success:
        print("⚠️ linprog:", res.message)
    print("C* =", float(c @ res.x))
    print("x* =", res.x)
    return res

c   = [2, 1, 2]
Age = np.array([[1, 1, 1],
                [2, 1, 3]])
bge = np.array([4, 6])

# Convertimos Ax >= b a -Ax <= -b
Aub = -Age
bub = -bge

res = solve_lp_min(c, A_ub=Aub, b_ub=bub)

C* = 5.0
x* = [0. 3. 1.]


##### **Tarea 1: Reformular un problema de minimización (1).**

Cambiar los RHS (*Right-Hand Side* o lado derecho) a $b=(5,7)$; resolver y comparar $C^*$ con el caso base.

In [48]:
# --- Sección 2: Minimización (Ax >= b -> -Ax <= -b) ---

import numpy as np
from scipy.optimize import linprog

def solve_lp_min(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs"):
    """
    Resuelve min c^T x en forma directa (<= , =).
    Retorna el objeto res de linprog; imprime C* y x*.
    """
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)
    res = linprog(c=c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)
    if not res.success:
        print("⚠️ linprog:", res.message)
    print("C* =", float(c @ res.x))
    print("x* =", res.x)
    return res

c   = [2, 1, 2]
Age = np.array([[1, 1, 1],
                [2, 1, 3]])
bge = np.array([5, 7])          # Cambio b=(5,7)

# Convertimos Ax >= b a -Ax <= -b
Aub = -Age
bub = -bge

res = solve_lp_min(c, A_ub=Aub, b_ub=bub)

C* = 6.0
x* = [0. 4. 1.]


##### **Tarea 2: Reformular un problema de minimización (2).**

Agregar la cota $x \le 3$; resolver e indicar qué restricciones quedan activas en el óptimo.

In [49]:
# --- Sección 2: Minimización (Ax >= b -> -Ax <= -b) ---

import numpy as np
from scipy.optimize import linprog

def solve_lp_min(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs"):
    """
    Resuelve min c^T x en forma directa (<= , =).
    Retorna el objeto res de linprog; imprime C* y x*.
    """
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)
    res = linprog(c=c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)
    if not res.success:
        print("⚠️ linprog:", res.message)
    print("C* =", float(c @ res.x))
    print("x* =", res.x)
    return res

c   = [2, 1, 2]
Age = np.array([[1, 1, 1],
                [2, 1, 3],
                [-1, 0, 0]])  # agregamos -1 debido a -1x + 0y +0z >= 3
bge = np.array([4, 6, -3])    # agregamos -3 debido a -x >= 3

# Convertimos Ax >= b a -Ax <= -b
Aub = -Age
bub = -bge

res = solve_lp_min(c, A_ub=Aub, b_ub=bub)

C* = 5.0
x* = [0. 3. 1.]


##### **Tarea 3: Resolver el dual explícito.**

Construir el dual:
$$
\begin{aligned}
\max\; P&=4u+6v \\
\text{s.a. }& u+2v\le 2,\\
& u+v\le 1,\\
& u+3v\le 2,\\
& u,v\ge 0,
\end{aligned}
$$

resolver con la función de $\max$ y verificar $C_{\min}=P_{\max}$.


In [50]:
# Maximización
import numpy as np
from scipy.optimize import linprog

# Función para maximizar c^T x
def solve_lp_maximizacion(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                          bounds=None, method="highs"):
    """
    Resuelve max c^T x mediante linprog (que minimiza -c^T x).
    Imprime C* y x*.
    """
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)  # x >= 0

    # linprog minimiza, así que usamos -c
    res = linprog(c=-c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)

    if not res.success:
        print("⚠️ linprog:", res.message)
        return res

    # Valor máximo real de la función objetivo
    Pmax = float(c @ res.x)
    print("Pmax =", Pmax)
    print("x* =", res.x)
    return res

# ---------------- Datos del problema ----------------
# Función a maximizar:
c = [4, 6]             # 4u + 6v

# Restricciones de desigualdad:
A_ub = np.array([
    [1, 2],             # u + 2v <= 2
    [1, 1],             # u + v  <= 1
    [1, 3]              # u + 3v <= 2
])
b_ub = np.array([2, 1, 2])

# Cota inferior: u, v >= 0
bounds = [(0, None), (0, None)]    # para ambas variables el minimo es 0 y no hay maximo

# Resolver
res = solve_lp_maximizacion(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)

# ---------------- Verificar Cmin = Pmax ----------------
# Convertimos a problema de minimización para comprobar
res_min = linprog(c=-np.array(c), A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs")
Cmin = -res_min.fun  # linprog minimiza -c^T x, así que -res.fun = max
print("\nVerificación: Cmin = Pmax =", Cmin)

Pmax = 5.0
x* = [0.5 0.5]

Verificación: Cmin = Pmax = 5.0


#### **3) Variantes (Two-Phase, Big-M, Dual Simplex)**

*Two-Phase* introduce variables artificiales para construir una base factible inicial (Fase 1) y luego optimizar el problema original (Fase 2). *Big-M* penaliza las artificiales en la función objetivo (FO), pero puede ser numéricamente sensible. *Dual Simplex* es útil cuando la base inicial es primal-infactible y dual-factible. En `linprog` (HiGHS), estrategias equivalentes se aplican internamente de forma robusta.


##### **Ejemplo: Restricción $\ge$ y $=$.**

$$
\begin{aligned}
\max\; z &= 3x_1 + x_2\\
\text{s.a.}\;& x_1 + x_2 \ge 4,\\
& x_1 + 2x_2 = 6,\\
& x_1, x_2 \ge 0.
\end{aligned}
$$

**Objetivo**   
- Modelar $\ge$ como $-\le$, la igualdad en $A_{eq}$,
- resolver con `linprog` y
- comentar qué restricciones quedan activas.

In [51]:
import numpy as np
from scipy.optimize import linprog

# Función para MAXIMIZAR una función lineal sujeta a restricciones
def solve_lp_max(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs", tol=1e-7):

    # Convierte el vector de coeficientes en un array
    c = np.array(c, dtype=float)

    # Si no se proporcionan cotas asume x >= 0
    if bounds is None:
        bounds = [(0, None)] * len(c)  # x_j >= 0

    # Linprog solo minimiza; entonces se invierte el signo de c para Maximizar
    res = linprog(c=-c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)

    if not res.success:
        print("⚠️ linprog:", res.message)
        return res  # Retorna el resultado aunque haya fallado

    # Calcula el máximo original de función objetivo
    z = float(c @ res.x)
    print("z* =", z)           # Imprime el valor máximo
    print("x* =", res.x)       # Imprime las variables óptimas

    # ---- 1) Verificar restricciones de desigualdad: A_ub x <= b_ub ----
    if A_ub is not None:
        slack = b_ub - A_ub @ res.x                 # Cuánto "sobran" las desigualdades
        print("slack (<=):", slack)
        # Detecta cuáles restricciones están activas
        active_ub = np.where(np.abs(slack) <= tol)[0].tolist()
        if active_ub:
            print("Restricciones <= activas (índices):", active_ub)
        else:
            print("No hay restricciones <= activas dentro de la tolerancia.")

    # ---- 2) Verificar restricciones de igualdad: A_eq x = b_eq ----
    if A_eq is not None:
        eq_resid = A_eq @ res.x - b_eq              # "Residuo" de las igualdades
        print("residuo (==):", eq_resid)
        active_eq = np.where(np.abs(eq_resid) <= tol)[0].tolist()
        if active_eq:
            print("Restricciones == satisfechas (índices):", active_eq)
        else:
            print("⚠️ Ninguna igualdad está dentro de la tolerancia (revisar).")

    # ---- 3) Verificar variables en cota inferior: x_j >= 0 ----
    active_bounds = np.where(res.x <= tol)[0].tolist()   # Variables =es a 0
    if active_bounds:
        print("Variables en cota inferior (x_j=0, activas):", active_bounds)
    else:
        print("Ninguna variable está exactamente en 0 (dentro de tolerancia).")

    return res


# ---- Datos del ejemplo ----
c    = [3, 1]                         # Coeficientes de la función objetivo
Aub  = -np.array([[1, 1]])            # Restricción: x1 + x2 >= 4 -> -x1 - x2 <= -4
bub  = -np.array([4])                 # Lado derecho de la desigualdad
Aeq  =  np.array([[1, 2]])            # Restricción de igualdad: x1 + 2*x2 = 6
beq  =  np.array([6])                 # Lado derecho de la igualdad

# Llamada a la función para resolver el problema de maximización
_ = solve_lp_max(c, A_ub=Aub, b_ub=bub, A_eq=Aeq, b_eq=beq, tol=1e-9)


z* = 18.0
x* = [6. 0.]
slack (<=): [2.]
No hay restricciones <= activas dentro de la tolerancia.
residuo (==): [0.]
Restricciones == satisfechas (índices): [0]
Variables en cota inferior (x_j=0, activas): [1]


##### **Tarea 1: Implementar el método de dos fases.**

Cambiar la igualdad a $x_1+2x_2=5$; resolver y comentar factibilidad y cambio en el óptimo.

In [52]:
# --- Sección 3: Variante (>= y =) con chequeo de restricciones activas ---

import numpy as np
from scipy.optimize import linprog

def solve_lp_max(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs", tol=1e-7):
    """
    Resuelve max c^T x (vía min -c^T x) y reporta restricciones activas:
      - Para A_ub x <= b_ub: activa si b_ub - A_ub x ~= 0
      - Para A_eq x  = b_eq: activa si |A_eq x - b_eq| ~= 0
      - Para no-negatividad: activa si x_j ~= 0
    """
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)  # x >= 0

    res = linprog(c=-c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)

    if not res.success:
        print("⚠️ linprog:", res.message)
        return res

    z = float(c @ res.x)
    print("z* =", z)
    print("x* =", res.x)

    # --- Chequeo de restricciones activas ---
    # 1) Desigualdades (A_ub x <= b_ub)
    if A_ub is not None:
        slack = b_ub - A_ub @ res.x
        print("slack (<=):", slack)
        active_ub = np.where(np.abs(slack) <= tol)[0].tolist()
        if active_ub:
            print("Restricciones <= activas (índices):", active_ub)
        else:
            print("No hay restricciones <= activas dentro de la tolerancia.")

    # 2) Igualdades (A_eq x = b_eq)
    if A_eq is not None:
        eq_resid = A_eq @ res.x - b_eq
        print("residuo (==):", eq_resid)
        active_eq = np.where(np.abs(eq_resid) <= tol)[0].tolist()
        if active_eq:
            print("Restricciones == satisfechas (índices):", active_eq)
        else:
            print("⚠️ Ninguna igualdad está dentro de la tolerancia (revisar).")

    # 3) No-negatividad (x >= 0) — activas si x_j ~= 0
    active_bounds = np.where(res.x <= tol)[0].tolist()
    if active_bounds:
        print("Variables en cota inferior (x_j=0, activas):", active_bounds)
    else:
        print("Ninguna variable está exactamente en 0 (dentro de tolerancia).")

    return res


# ---- Datos del ejemplo ----
c    = [3, 1]
Aub  = -np.array([[1, 1]])   # x1 + x2 >= 4  ->  -x1 - x2 <= -4
bub  = -np.array([4])
Aeq  =  np.array([[1, 2]])   # x1 + 2 x2 = 5
beq  =  np.array([5])

_ = solve_lp_max(c, A_ub=Aub, b_ub=bub, A_eq=Aeq, b_eq=beq, tol=1e-9)

z* = 15.0
x* = [5. 0.]
slack (<=): [1.]
No hay restricciones <= activas dentro de la tolerancia.
residuo (==): [0.]
Restricciones == satisfechas (índices): [0]
Variables en cota inferior (x_j=0, activas): [1]


##### **Tarea 2: Cambio de desigualdad.**

Sustituir $x_1 + x_2 \ge 4$ por $x_1+3x_2\ge 5$, en el problema del ejemplo.


In [53]:
# --- Sección 3: Variante (>= y =) con chequeo de restricciones activas ---

import numpy as np
from scipy.optimize import linprog

def solve_lp_max(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs", tol=1e-7):
    """
    Resuelve max c^T x (vía min -c^T x) y reporta restricciones activas:
      - Para A_ub x <= b_ub: activa si b_ub - A_ub x ~= 0
      - Para A_eq x  = b_eq: activa si |A_eq x - b_eq| ~= 0
      - Para no-negatividad: activa si x_j ~= 0
    """
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)  # x >= 0

    res = linprog(c=-c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)

    if not res.success:
        print("⚠️ linprog:", res.message)
        return res

    z = float(c @ res.x)
    print("z* =", z)
    print("x* =", res.x)

    # --- Chequeo de restricciones activas ---
    # 1) Desigualdades (A_ub x <= b_ub)
    if A_ub is not None:
        slack = b_ub - A_ub @ res.x
        print("slack (<=):", slack)
        active_ub = np.where(np.abs(slack) <= tol)[0].tolist()
        if active_ub:
            print("Restricciones <= activas (índices):", active_ub)
        else:
            print("No hay restricciones <= activas dentro de la tolerancia.")

    # 2) Igualdades (A_eq x = b_eq)
    if A_eq is not None:
        eq_resid = A_eq @ res.x - b_eq
        print("residuo (==):", eq_resid)
        active_eq = np.where(np.abs(eq_resid) <= tol)[0].tolist()
        if active_eq:
            print("Restricciones == satisfechas (índices):", active_eq)
        else:
            print("⚠️ Ninguna igualdad está dentro de la tolerancia (revisar).")

    # 3) No-negatividad (x >= 0) — activas si x_j ~= 0
    active_bounds = np.where(res.x <= tol)[0].tolist()
    if active_bounds:
        print("Variables en cota inferior (x_j=0, activas):", active_bounds)
    else:
        print("Ninguna variable está exactamente en 0 (dentro de tolerancia).")

    return res


# ---- Datos del ejemplo ----
c    = [3, 1]
Aub  = -np.array([[1, 3]])   # x1 + 3x2 >= 5  ->  -x1 - x2 <= -4
bub  = -np.array([5])
Aeq  =  np.array([[1, 2]])   # x1 + 2 x2 = 5
beq  =  np.array([5])

_ = solve_lp_max(c, A_ub=Aub, b_ub=bub, A_eq=Aeq, b_eq=beq, tol=1e-9)

z* = 15.0
x* = [5. 0.]
slack (<=): [0.]
Restricciones <= activas (índices): [0]
residuo (==): [0.]
Restricciones == satisfechas (índices): [0]
Variables en cota inferior (x_j=0, activas): [1]


#### **4) Problema de transporte (estructura y código)**

En el **transporte** balanceado, dadas ofertas $(\text{oferta}_i)$ y demandas $(\text{demanda}_j)$ con $\sum_i \text{oferta}_i = \sum_j \text{demanda}_j$, y costos unitarios $c_{ij}$, se minimiza
$$
\min\; Z = \sum_{i=1}^m\sum_{j=1}^n c_{ij} x_{ij}
\quad \text{s.a.}\quad
\sum_{j=1}^n x_{ij} = \text{oferta}_i,\;
\sum_{i=1}^m x_{ij} = \text{demanda}_j,\;
x_{ij}\ge 0.
$$
Vectorizando por filas, construimos $A_{eq}$ con $m$ ecuaciones de oferta y $n$ de demanda.

##### **Ejemplo: Envío de altavoces ($2\times3$)**

**Descripción del problema:**  
Una empresa produce altavoces en dos plantas (I y II) y debe distribuirlos hacia tres depósitos (A, B y C).  
Los costos unitarios de transporte (en dólares) son:
$$
\begin{array}{c|ccc}
 & A & B & C \\ \hline
\text{I} & 20 & 8 & 10 \\
\text{II} & 12 & 22 & 18
\end{array}
$$

La capacidad de producción (oferta) y las necesidades (demanda) son:
$$
\text{Oferta} = (350,\, 550), \qquad
\text{Demanda} = (200,\, 300,\, 400).
$$

**Objetivos:**  
- Construir automáticamente las estructuras del modelo en Python:  
  - El vector de costos $c$ (aplanando la matriz de costos).  
  - La matriz de restricciones $A_{eq}$ y el vector $b_{eq}$ que representen las igualdades de oferta y demanda.  
- Resolver el problema de transporte como un $\min$ con `linprog`.  
- Imprimir el plan de envío $x$ en forma matricial ($2\times 3$).  
- Analizar qué arcos $x_{ij}$ resultan **positivos** en la solución óptima, y discutir por qué esos son los seleccionados.


In [54]:
# --- Sección 4: Transporte (balanceado: Suma de ofertas = Suma de demandas) ---

import numpy as np
from scipy.optimize import linprog

def build_transport_lp(cost, supply, demand):
    """
    Devuelve c (aplanado por filas), A_eq, b_eq, bounds (x_ij >= 0).
    """
    cost   = np.asarray(cost, dtype=float)
    supply = np.asarray(supply, dtype=float)
    demand = np.asarray(demand, dtype=float)
    m, n = cost.shape
    c = cost.flatten()
    # Oferta: m ecuaciones
    A_sup = np.zeros((m, m*n))
    for i in range(m):
        A_sup[i, i*n:(i+1)*n] = 1.0
    # Demanda: n ecuaciones
    A_dem = np.zeros((n, m*n))
    for j in range(n):
        A_dem[j, j::n] = 1.0
    A_eq = np.vstack([A_sup, A_dem])
    b_eq = np.concatenate([supply, demand])
    bounds = [(0, None)] * (m*n)
    return c, A_eq, b_eq, bounds

def solve_lp_min(c, A_eq=None, b_eq=None, bounds=None, method="highs"):
    res = linprog(c=c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method=method)
    if not res.success:
        print("⚠️ linprog:", res.message)
    print("Costo mínimo =", float(res.fun))
    return res

cost   = np.array([[20, 8, 10],
                   [12,22,18]])
supply = np.array([350, 550])
demand = np.array([200, 300, 400])

c, Aeq, beq, bnds = build_transport_lp(cost, supply, demand)
res = solve_lp_min(c, A_eq=Aeq, b_eq=beq, bounds=bnds)
print("Plan (matriz):\n", res.x.reshape(cost.shape))

Costo mínimo = 11600.0
Plan (matriz):
 [[  0. 300.  50.]
 [200.   0. 350.]]


##### **Tarea 1: Cambio de costo.**

 Cambiar el costo (I→B) de 8 a 9; resolver y comparar costo total y plan con el caso base.

In [55]:
# --- Sección 4: Transporte (balanceado: Suma de ofertas = Suma de demandas) ---

import numpy as np
from scipy.optimize import linprog

def build_transport_lp(cost, supply, demand):
    """
    Devuelve c (aplanado por filas), A_eq, b_eq, bounds (x_ij >= 0).
    """
    cost   = np.asarray(cost, dtype=float)
    supply = np.asarray(supply, dtype=float)
    demand = np.asarray(demand, dtype=float)
    m, n = cost.shape
    c = cost.flatten()
    # Oferta: m ecuaciones
    A_sup = np.zeros((m, m*n))
    for i in range(m):
        A_sup[i, i*n:(i+1)*n] = 1.0
    # Demanda: n ecuaciones
    A_dem = np.zeros((n, m*n))
    for j in range(n):
        A_dem[j, j::n] = 1.0
    A_eq = np.vstack([A_sup, A_dem])
    b_eq = np.concatenate([supply, demand])
    bounds = [(0, None)] * (m*n)
    return c, A_eq, b_eq, bounds

def solve_lp_min(c, A_eq=None, b_eq=None, bounds=None, method="highs"):
    res = linprog(c=c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method=method)
    if not res.success:
        print("⚠️ linprog:", res.message)
    print("Costo mínimo =", float(res.fun))
    return res

cost   = np.array([[20, 9, 10],    # se cambia [20, 8, 10] por [20, 9, 10]
                   [12,22,18]])
supply = np.array([350, 550])
demand = np.array([200, 300, 400])

c, Aeq, beq, bnds = build_transport_lp(cost, supply, demand)
res = solve_lp_min(c, A_eq=Aeq, b_eq=beq, bounds=bnds)
print("Plan (matriz):\n", res.x.reshape(cost.shape))

Costo mínimo = 11900.0
Plan (matriz):
 [[  0. 300.  50.]
 [200.   0. 350.]]


##### **Tarea 2: Caso no balanceado (Transporte con origen ficticio).**

**Descripción del problema:**  
Un sistema de transporte tiene la siguiente matriz de costos unitarios (\$):
$$
\begin{array}{c|ccc}
     & D_1 & D_2 & D_3 \\
\hline
O_1 & 4   & 6   & 8 \\
O_2 & 5   & 4   & 3
\end{array}
$$

Las ofertas y demandas son:
- Oferta: $O_1 = 100$, $O_2 = 120$  
- Demanda: $D_1 = 80$, $D_2 = 70$, $D_3 = 90$  

Observá que la suma de las ofertas ($220$) no coincide con la suma de las demandas ($240$).  

**Objetivos:**  
- Reformular el problema para que la oferta y la demanda sean iguales.  
- Para ello, introducí un **origen ficticio** que cubra el faltante.  
  - *Pista: podés agregar en la lista de ofertas (`supply`) una entrada adicional de 20 unidades, con costos nulos a todos los destinos.*
- Implementar el modelo en Python y resolverlo con `linprog`.  
- Comparar el costo obtenido con y sin el origen ficticio.  


In [56]:
# --- Sección 4: Transporte (balanceado: Suma de ofertas = Suma de demandas) ---

import numpy as np
from scipy.optimize import linprog

def build_transport_lp(cost, supply, demand):
    """
    Construye los componentes de un modelo de transporte de linprog.
    Devuelve c (aplanado por filas), A_eq, b_eq, bounds (x_ij >= 0).
    """
    cost   = np.asarray(cost, dtype=float)
    supply = np.asarray(supply, dtype=float)
    demand = np.asarray(demand, dtype=float)
    m, n = cost.shape

    # 1. Vector de costos c (aplanado por filas)
    c = cost.flatten()

    # 2. Matriz de igualdad A_eq
    # Restricciones de oferta (m ecuaciones)
    A_sup = np.zeros((m, m*n))
    for i in range(m):
        A_sup[i, i*n:(i+1)*n] = 1.0

    # Restricciones de demanda (n ecuaciones)
    A_dem = np.zeros((n, m*n))
    for j in range(n):
        A_dem[j, j::n] = 1.0

    A_eq = np.vstack([A_sup, A_dem])

    # 3. Vector de igualdad b_eq
    b_eq = np.concatenate([supply, demand])

    # 4. Límites de las variables (x_ij >= 0)
    bounds = [(0, None)] * (m*n)

    return c, A_eq, b_eq, bounds

def solve_lp_min(c, A_eq=None, b_eq=None, bounds=None, method="highs"):
    """
    Resuelve el problema de minimización y reporta el resultado.
    """
    res = linprog(c=c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")
    if not res.success:
        print("⚠️ linprog:", res.message)
    print("Costo mínimo =", float(res.fun))
    return res

print("--- Caso 1: Problema no balanceado (sin origen ficticio) ---")
cost_orig = np.array([[4, 5, 8],
                      [6, 4, 3]])
supply_orig = np.array([100, 120])
demand_orig = np.array([80, 70, 90])

print(f"Suma de oferta: {np.sum(supply_orig)}")
print(f"Suma de demanda: {np.sum(demand_orig)}")
if np.sum(supply_orig) == np.sum(demand_orig):
    print("El problema está balanceado.")
    c_orig, Aeq_orig, beq_orig, bnds_orig = build_transport_lp(cost_orig, supply_orig, demand_orig)
    res_orig = solve_lp_min(c_orig, A_eq=Aeq_orig, b_eq=beq_orig, bounds=bnds_orig)
else:
    print("El problema no está balanceado. La solución será 'infeasible'.")
    # --- 2. Caso con origen ficticio (balanceado) ---
    print("\n--- Caso 2: Problema balanceado con origen ficticio ---")

    # Se calcula el faltante para la oferta
    falta = np.sum(demand_orig) - np.sum(supply_orig)
    print(f"El faltante de oferta es: {falta}")

    # Se crea la nueva matriz de costos con una fila adicional de ceros
    # El origen ficticio tiene costo cero para todos los destinos
    cost_fict = np.vstack([cost_orig, [0, 0, 0]])

    # Se crea el nuevo vector de ofertas con la oferta ficticia
    supply_fict = np.append(supply_orig, falta)

    print(f"Nueva suma de oferta: {np.sum(supply_fict)}")
    print(f"Nueva suma de demanda: {np.sum(demand_orig)}")
    print("El problema ahora está balanceado.")

    # Se construye y resuelve el modelo balanceado
    c, Aeq, beq, bnds = build_transport_lp(cost_fict, supply_fict, demand_orig)
    res_fict = solve_lp_min(c, A_eq=Aeq, b_eq=beq, bounds=bnds)

# Se muestra el plan de transporte como una matriz
print("Plan (matriz):\n", res_fict.x.reshape(cost_fict.shape))


--- Caso 1: Problema no balanceado (sin origen ficticio) ---
Suma de oferta: 220
Suma de demanda: 240
El problema no está balanceado. La solución será 'infeasible'.

--- Caso 2: Problema balanceado con origen ficticio ---
El faltante de oferta es: 20
Nueva suma de oferta: 240
Nueva suma de demanda: 240
El problema ahora está balanceado.
Costo mínimo = 810.0
Plan (matriz):
 [[80. 20.  0.]
 [ 0. 30. 90.]
 [ 0. 20.  0.]]


Comparación de los resultados

El problema de transporte original es no balanceado porque la oferta total es 220 que es menor que la demanda total de 240.

Cuando intentamos resolver un problema de transporte no balanceado directamente con linprog (utilizando restricciones de igualdad), el algoritmo no encuentra una solución y lo reporta.

Esto se da ya que las restricciones de oferta y demanda no pueden ser satisfechas al mismo tiempo.

Para resolver este problema, se introduce un origen ficticio.
Este origen representa el faltante de la oferta de nuestro problema (20 unidades) y tiene un costo de transporte de cero a todos los destinos, ya que no se enviara ningún producto físico desde allí.
Las unidades "transportadas" desde **este origen ficticio** a cada destino representan la ***cantidad de demanda insatisfecha en ese destino***.

Al resolver el problema con el origen ficticio, el modelo se convierte en "balanceado" y linprog puede encontrar una solución óptima.

El costo mínimo obtenido de esta solución es el costo mínimo de transporte de las unidades reales (recordando que el transporte desde el origen ficcticio tiene valor 0).

El plan de transporte resultante (res.x en forma de matriz) mostrará las unidades transportadas entre los orígenes reales y las demandas, así como las unidades que no pudieron ser satisfechas desde el origen ficticio.

En resumen: La diferencia es que el modelo original no tiene solución (es infactible), mientras que el modelo rediseñado con el origen ficticio si tiene una solución y nos proporciona el costo mínimo real del transporte.


##### **Tarea 3: Instancia aleatoria balanceada ($5\times 7$).**


Generar costos enteros en $[5,30]$, ofertas/demandas enteras balanceadas; resolver y reportar el costo mínimo.

In [57]:
# --- Sección 4: Transporte (balanceado: Suma de ofertas = Suma de demandas) ---

import numpy as np
from scipy.optimize import linprog

def build_transport_lp(cost, supply, demand):
    """
    Construye los componentes de un modelo de transporte de linprog.
    Devuelve c (aplanado por filas), A_eq, b_eq, bounds (x_ij >= 0).
    """
    cost = np.asarray(cost, dtype=float)
    supply = np.asarray(supply, dtype=float)
    demand = np.asarray(demand, dtype=float)
    m, n = cost.shape

    # 1. Vector de costos c (aplanado por filas)
    c = cost.flatten()

    # 2. Matriz de igualdad A_eq
    # Restricciones de oferta (m ecuaciones)
    A_sup = np.zeros((m, m*n))
    for i in range(m):
        A_sup[i, i*n:(i+1)*n] = 1.0

    # Restricciones de demanda (n ecuaciones)
    A_dem = np.zeros((n, m*n))
    for j in range(n):
        A_dem[j, j::n] = 1.0

    A_eq = np.vstack([A_sup, A_dem])

    # 3. Vector de igualdad b_eq
    # FIX: Se aplanan los arrays de oferta y demanda para asegurar que b_eq sea 1D.
    b_eq = np.concatenate([supply.flatten(), demand.flatten()])

    # 4. Límites de las variables (x_ij >= 0)
    bounds = [(0, None)] * (m*n)

    return c, A_eq, b_eq, bounds

def solve_lp_min(c, A_eq=None, b_eq=None, bounds=None, method="highs"):
    """
    Resuelve el problema de minimización y reporta el resultado.
    """
    res = linprog(c=c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")
    if not res.success:
        print("⚠️ linprog:", res.message)
    print("Costo mínimo =", float(res.fun))
    return res

# --- Sección de prueba modificada para un mejor manejo de datos ---

print("--- Caso 1: Problema no balanceado (sin origen ficticio) ---")

# Generar datos de forma robusta
m_rand = 5  # Número de orígenes
n_rand = 7  # Número de destinos

# Generar valores de oferta y demanda
supply_orig = np.random.randint(5, 50, size=m_rand)
demand_orig = np.random.randint(5, 50, size=n_rand)
cost_orig = np.random.randint(5, 31, size=(m_rand, n_rand))

suma_oferta = np.sum(supply_orig)
suma_demanda = np.sum(demand_orig)

print(f"Suma de oferta: {suma_oferta}")
print(f"Suma de demanda: {suma_demanda}")

if suma_oferta == suma_demanda:
    print("El problema está balanceado.")
    c_orig, Aeq_orig, beq_orig, bnds_orig = build_transport_lp(cost_orig, supply_orig, demand_orig)
    res_orig = solve_lp_min(c_orig, A_eq=Aeq_orig, b_eq=beq_orig, bounds=bnds_orig)
    print("Plan (matriz):\n", res_orig.x.reshape(cost_orig.shape))
else:
    print("El problema no está balanceado. La solución será 'infeasible' sin ajuste.")

    # Se calcula la diferencia entre oferta y demanda
    diferencia = suma_demanda - suma_oferta

    # Si la demanda es mayor, se agrega un origen ficticio.
    if diferencia > 0:
        falta = diferencia
        print(f"El faltante de oferta es: {falta}. Se agregará un origen ficticio.")
        # Se crea la nueva matriz de costos
        cost_fict = np.vstack([cost_orig, np.zeros((1, n_rand))])
        # Se crea el nuevo vector de ofertas con la oferta ficticia
        supply_fict = np.append(supply_orig, falta)
        demand_fict = demand_orig

    # Si la oferta es mayor, se agrega un destino ficticio.
    else:
        exceso = -diferencia
        print(f"El exceso de oferta es: {exceso}. Se agregará un destino ficticio.")
        # Se crea la nueva matriz de costos
        cost_fict = np.hstack([cost_orig, np.zeros((m_rand, 1))])
        # Se crea el nuevo vector de demandas con la demanda ficticia
        demand_fict = np.append(demand_orig, exceso)
        supply_fict = supply_orig

    print(f"Nueva suma de oferta: {np.sum(supply_fict)}")
    print(f"Nueva suma de demanda: {np.sum(demand_fict)}")
    print("El problema ahora está balanceado.")

    # Construye y resuelve el modelo balanceado
    c, Aeq, beq, bnds = build_transport_lp(cost_fict, supply_fict, demand_fict)
    res_fict = solve_lp_min(c, A_eq=Aeq, b_eq=beq, bounds=bnds)

    # Se muestra el plan de transporte
    print("Plan (matriz):\n", res_fict.x.reshape(cost_fict.shape))



--- Caso 1: Problema no balanceado (sin origen ficticio) ---
Suma de oferta: 125
Suma de demanda: 165
El problema no está balanceado. La solución será 'infeasible' sin ajuste.
El faltante de oferta es: 40. Se agregará un origen ficticio.
Nueva suma de oferta: 165
Nueva suma de demanda: 165
El problema ahora está balanceado.
Costo mínimo = 1611.0
Plan (matriz):
 [[25.  0.  0.  0. 18.  3.  0.]
 [ 0.  0.  0.  0.  0.  7.  0.]
 [ 0.  0.  0.  0.  5.  0.  0.]
 [ 0. 16.  0. 21.  0.  4.  7.]
 [ 0. 19.  0.  0.  0.  0.  0.]
 [ 0.  0. 11.  0.  0. 29.  0.]]


#### **5) Problema de asignación (algoritmo Húngaro)**

En **asignación** $n\times n$ se busca $\min \sum c_{ij} x_{ij}$ s.a. cada agente hace exactamente una tarea y cada tarea recibe exactamente un agente (variables binarias). Su estructura permite resolverlo eficientemente mediante el *algoritmo Húngaro* (Kuhn-Munkres). Para maximizar beneficios, se convierte a minimización restando de una constante $M\ge \max B_{ij}$.

##### **Ejemplo: $3\times 3$.**

$$
\begin{array}{c|ccc}
 & A & B & C \\ \hline
\text{T1} & 14 & 5 & 8 \\
\text{T2} & 2 & 12 & 6 \\
\text{T3} & 7 & 8 & 3
\end{array}
$$
**Objetivo:**   
Aplicar `linear_sum_assignment` para obtener la asignación óptima y verificar el costo mínimo.

In [58]:
# --- Sección 5: Asignación (algoritmo Húngaro) ---

import numpy as np
from scipy.optimize import linear_sum_assignment

cost = np.array([[14, 5, 8],
                 [ 2,12, 6],
                 [ 7, 8, 3]])

rows, cols = linear_sum_assignment(cost)
print("Asignación (i->j):", list(zip(rows, cols)))
print("Costo mínimo =", cost[rows, cols].sum())

Asignación (i->j): [(np.int64(0), np.int64(1)), (np.int64(1), np.int64(0)), (np.int64(2), np.int64(2))]
Costo mínimo = 10


##### **Tarea 1: Salida legible en el problema de asignación.**

**Descripción del problema:**  
Repetir el ejemplo de asignación con 3 trabajadores y 3 tareas.  
Actualmente, `linear_sum_assignment` devuelve la salida como índices base-0 (por ejemplo `[(0,1),(1,0),(2,2)]`), lo cual puede ser confuso.  

**Objetivos:**  
- Modificar el código para mostrar la salida en forma legible, listando los pares $(\text{trabajador} \to \text{tarea})$.  
- Verificar explícitamente la suma de los costos asociados a la asignación óptima.


In [59]:
# --- Sección 5: Asignación (algoritmo Húngaro) ---

import numpy as np
from scipy.optimize import linear_sum_assignment

# Matriz de costos: filas = trabajadores, columnas = tareas
cost = np.array([[14, 5, 8],
                 [ 2,12, 6],
                 [ 7, 8, 3]])

# Aplicar el algoritmo Húngaro para encontrar la asignación de costo mínimo
row_ind, col_ind = linear_sum_assignment(cost)
# La función linear_sum_assignment trae los índices de fila (trabajadores) y columna (tareas)

# Generar salida legible ---
print("La asignación óptima es:")
for i in range(len(row_ind)):
    trabajador = row_ind[i]
    tarea = col_ind[i]
    costo_individual = cost[trabajador, tarea]
    print(f"Trabajador {trabajador + 1} -> Tarea {tarea + 1} (Costo: {costo_individual})") # se agrega 1 al numero de trabajador y tarea ya q los indices comienzan con 0

# Calcular el costo total sumando los costos individuales de la asignación
costo_total = cost[row_ind, col_ind].sum()
print("\nCosto mínimo total =", costo_total)

La asignación óptima es:
Trabajador 1 -> Tarea 2 (Costo: 5)
Trabajador 2 -> Tarea 1 (Costo: 2)
Trabajador 3 -> Tarea 3 (Costo: 3)

Costo mínimo total = 10


##### **Tarea 2: Maximización de beneficio.**

Generar una matriz $4\times 4$ de beneficios enteros en $[10,50]$; convertir a costos por $M-\text{benef}$ con $M=\max$ de la matriz; resolver y reportar el beneficio total.

In [60]:
import numpy as np
from scipy.optimize import linear_sum_assignment

# Generar matriz de beneficios 4x4 con valores enteros en [10, 50]
np.random.seed(42)  # Para resultados reproducibles
beneficios = np.random.randint(10, 51, size=(4, 4))
print("Matriz de Beneficios:")
print(beneficios)

# Convertir matriz de beneficios en matriz de costos
M = np.max(beneficios)
costos = M - beneficios
print("\nMatriz de Costos (transformada para minimización):")
print(costos)

# Resolver la asignación de costo mínimo
row_ind, col_ind = linear_sum_assignment(costos)
print("La asignación óptima es:")
for i in range(len(row_ind)):
    trabajador = row_ind[i]
    tarea = col_ind[i]
    beneficio_individual = beneficios[trabajador, tarea]
    print(f"Trabajador {trabajador + 1} -> Tarea {tarea + 1} (Beneficio: {beneficio_individual})") # se agrega 1 al numero de trabajador y tarea ya q los indices comienzan con 0

# Calcular el costo total sumando los costos individuales de la asignación
beneficio_total = beneficios[row_ind, col_ind].sum()
print("\nBeneficio maximo total =", beneficio_total)


Matriz de Beneficios:
[[48 38 24 17]
 [30 48 28 32]
 [20 20 33 45]
 [49 33 12 31]]

Matriz de Costos (transformada para minimización):
[[ 1 11 25 32]
 [19  1 21 17]
 [29 29 16  4]
 [ 0 16 37 18]]
La asignación óptima es:
Trabajador 1 -> Tarea 3 (Beneficio: 24)
Trabajador 2 -> Tarea 2 (Beneficio: 48)
Trabajador 3 -> Tarea 4 (Beneficio: 45)
Trabajador 4 -> Tarea 1 (Beneficio: 49)

Beneficio maximo total = 166


##### **Tarea 3: Escala grande.**

Resolver una instancia $n=100$ con costos enteros en $[1,100]$; reportar costo mínimo y tiempo de ejecución.

In [61]:
import numpy as np
import time
from scipy.optimize import linear_sum_assignment

# Tamaño de la matriz
n = 100

# Generar matriz de costos de nxn con valores enteros en [1, 100]
costos = np.random.randint(1, 101, size=(n, n))

# Inicia tiempo de ejecución
inicio_tiempo = time.time()

# Resolver problema
row_ind, col_ind = linear_sum_assignment(costos)

# Fin tiempo de ejecucion
fin_tiempo = time.time()
tiempo_ejecucion = fin_tiempo - inicio_tiempo

# Calcular costo mínimo
costo_minimo = costos[row_ind, col_ind].sum()

print(f"Tamaño de la matriz: {n}x{n}")
print(f"Costo mínimo total: {costo_minimo}")
print(f"Tiempo de ejecución: {tiempo_ejecucion:.6f} segundos")

Tamaño de la matriz: 100x100
Costo mínimo total: 216
Tiempo de ejecución: 0.000439 segundos


#### **Notas finales para el laboratorio**
1. Verifique siempre $x\ge 0$ y las folgas/holguras en las restricciones activas.
2. En Simplex, al finalizar: *variables básicas* toman el valor del T.I.; *no básicas* quedan en 0.
3. Para $\min$ con $A x \ge b$, considerar resolver el **dual** si la conversión directa resulta incómoda o inestable.
4. Experimente con dimensiones mayores para observar el rendimiento de HiGHS en `linprog`.